# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_csv = "weather_data.csv"
weather_data_df = pd.read_csv(weather_data_csv)
weather_data_df


,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arraial do Cabo,-22.9661,-42.0278,294.07,68,75,2.57,BR,1633729266
1,1,Māndleshwar,22.1833,75.6667,300.37,65,65,0.50,IN,1633729268
2,2,Broome,42.2506,-75.8330,294.24,70,1,0.45,US,1633729270
3,3,Punta Arenas,-53.1500,-70.9167,282.21,40,20,15.95,CL,1633729040
4,4,Margate,51.3813,1.3862,285.86,82,7,1.34,GB,1633729273
...,...,...,...,...,...,...,...,...,...,...
569,569,Beira,-19.8436,34.8389,292.09,88,0,2.68,MZ,1633730112
570,570,Dwārka,22.2394,68.9678,301.52,77,14,2.78,IN,1633730113
571,571,Cururupu,-1.8283,-44.8683,298.42,84,18,1.69,BR,1633730114
572,572,Evanston,42.0411,-87.6901,293.90,83,61,2.15,US,1633730017


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
# Store "Lat" and "Lng" into locations
locations = weather_data_df[['Lat', 'Lon']].astype(float)

# Store 'Humidity' into humidity 
humidity = weather_data_df['Humidity'].astype(float)


In [5]:
# Create a humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Narrow dow weather data to a few cities with warm temp and high humidity

good_humidity = weather_data_df[(weather_data_df['Humidity'].between(35, 55))].dropna()
good_temp = good_humidity[(good_humidity['Temp'].between (296, 304))].dropna()
best_weather = good_temp[(good_temp["Wind Speed"].between(3, 20))].dropna()


8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "Hotel",
    "key": g_key
    }

# Iterate through 
for index, row in best_weather.iterrows():
    # get lat, lon from df
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    address = response.json()
    try:
        best_weather['Hotel Name'] = address["results"][0]["name"]
    except:    
        print('skipping error')
        pass

#  View the hotel options   
best_weather


,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,74,Assiut,27.1810,31.1837,297.25,47,0,3.73,EG,1633729376,Dedougou
194,194,Nouadhibou,20.9310,-17.0347,300.14,54,52,7.20,MR,1633729555,Dedougou
207,207,Najrān,17.4924,44.1277,299.51,46,100,3.38,SA,1633729575,Dedougou
217,217,Estelle,29.8458,-90.1067,303.10,48,20,3.60,US,1633729590,Dedougou
300,300,Makakilo City,21.3469,-158.0858,300.56,51,75,7.72,US,1633729717,Dedougou
442,442,Sur,22.5667,59.5289,301.15,55,2,3.52,OM,1633729827,Dedougou
507,507,San Patricio,28.0170,-97.5169,302.90,43,0,5.26,US,1633730019,Dedougou
548,548,Dédougou,12.4667,-3.4667,301.75,48,100,3.25,BF,1633730079,Dedougou


In [33]:
# List of humidity for the heatmap
humidity_level = best_weather["Humidity"].tolist()

# list of hotels for the hotel markers
hotel_name = best_weather['Hotel Name'].tolist()


In [34]:
# Note: Do not change the code in this cell 

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in best_weather.iterrows()]
hotel_locations = best_weather[["Lat", "Lon"]].astype(float)


In [35]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(hotel_locations, weights= humidity_level,
                                dissipating=False, max_intensity=100,
                                point_radius=2)

fig.add_layer(heat_layer)


In [36]:
# Creat Bank Marker Map
hotel_layer = gmaps.symbol_layer(
hotel_locations, fill_color = 'rgba(100,150,0,0.4)',
    stroke_color= 'rgba(0,150,0,0.4)', scale=2,
    info_box_content=[f"Hotel Name:{hotel}" for hotel in hotel_name]
)

fig = gmaps.figure()
fig.add_layer(hotel_layer)


In [37]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig


Figure(layout=FigureLayout(height='420px'))